# MINT and DSSAT Analysis

Since a complete and working version of DSSAT has been provided to Jataware, it presents a good opportunity to compare the inputs and outputs that are present in the UFL provided version of DSSAT and the metadata existing about DSSAT in the MINT Model and Data Catalogs.

This notebook does the following:

1. [Gets all models from MINT](#Get-all-Models)
2. [Obtain DSSAT model](#Obtain-DSSAT-Model)
3. [Get DSSAT input/output files](#Get-DSSAT-Variable-Representations)
4. [Get DSSAT variable information](#Get-DSSAT-Variable-Information)
5. [Show DSSAT variables per input file](#DSSAT-Inputs-by-File)
6. [Show DSSAT variables per output file](#DSSAT-Outputs-by-File)

## Get all Models
Here we pull all models in the MINT Model catalog using the Garlic endpoint:

In [101]:
import pandas as pd

In [9]:
import requests

params = (
    ('endpoint', 'https://endpoint.mint.isi.edu/ds/query'),
)

response = requests.get('https://query.mint.isi.edu/api/mintproject/MINT-ModelCatalogQueries/getModels', params=params)

In [17]:
models = response.json()['results']['bindings']

In [20]:
len(models)

9

In [22]:
for m in models:
    print(m['label']['value'])

WGEN
Aggregate crop supply response model
Kimetrica land use model
DSSAT
Modflow
PIHM
TopoFlow
Cycles
ClimComp


In [127]:
models[0]

{'categories': {'type': 'literal', 'value': 'Weather'},
 'desc': {'type': 'literal',
  'value': 'WGEN is a globally applicable weather generator'},
 'doc': {'type': 'literal', 'value': 'https://arve-research.github.io/gwgen/'},
 'label': {'type': 'literal', 'value': 'WGEN'},
 'model': {'type': 'uri', 'value': 'https://w3id.org/mint/instance/WGEN'},
 'versions': {'type': 'literal',
  'value': 'https://w3id.org/mint/instance/wgen_1.0.1'}}

## Obtain DSSAT Model
Next, we filter for just the DSSAT model

In [23]:
for m in models:
    if m['label']['value'] == 'DSSAT':
        dssat = m

In [24]:
dssat

{'categories': {'type': 'literal', 'value': 'Agriculture'},
 'desc': {'type': 'literal',
  'value': 'The Decision Support System for Agrotechnology Transfer (DSSAT) comprises dynamic crop growth simulation model for over 40 crops. The model simulates growth; development; and yield as a function of the soil-plant-atmosphere dynamics.'},
 'doc': {'type': 'literal', 'value': 'https://dssat.net'},
 'label': {'type': 'literal', 'value': 'DSSAT'},
 'model': {'type': 'uri', 'value': 'https://w3id.org/mint/instance/DSSAT'},
 'versions': {'type': 'literal',
  'value': 'https://w3id.org/mint/instance/DSSAT_4.7'}}

In [54]:
dssat_uri = dssat['model']['value']

## Get DSSAT Variable Representations
Now we can obtain the variable representations for DSSAT. This includes the input and output files associated with the DSSAT model.

In [55]:
import requests

params = (
    ('model', dssat_uri),
    ('endpoint', 'https://endpoint.mint.isi.edu/ds/query'),
)

response = requests.get('https://query.mint.isi.edu/api/mintproject/MINT-ModelCatalogQueries/getVariablePresentationsForModel', params=params)

In [56]:
var_rep = response.json()['results']['bindings'][0]

In [57]:
var_rep

{'compLoc': {'type': 'literal',
  'value': 'https://github.com/KnowledgeCaptureAndDiscovery/MINT-WorkflowDomain/raw/master/WINGSWorkflowComponents/DSSAT_config_simple/DSSAT_config_simple.zip'},
 'config': {'type': 'uri',
  'value': 'https://w3id.org/mint/instance/DSSAT_cfg_simple'},
 'configDesc': {'type': 'literal',
  'value': 'DSSAT simple working configuration with explicit outputs'},
 'input_files': {'type': 'literal',
  'value': 'https://w3id.org/mint/instance/DSSAT_weatherFile, https://w3id.org/mint/instance/DSSAT_simulationFile, https://w3id.org/mint/instance/DSSAT_sol'},
 'output_files': {'type': 'literal',
  'value': 'https://w3id.org/mint/instance/DSSAT_summary'},
 'version': {'type': 'uri',
  'value': 'https://w3id.org/mint/instance/DSSAT_4.7'}}

## Get DSSAT Variable Information
Here we parse the input and output files and define a function `get_variables` for retrieving the variables associated with each file.

In [79]:
input_files = var_rep['input_files']['value'].split(', ')

In [80]:
input_files

['https://w3id.org/mint/instance/DSSAT_weatherFile',
 'https://w3id.org/mint/instance/DSSAT_simulationFile',
 'https://w3id.org/mint/instance/DSSAT_sol']

In [81]:
output_files = var_rep['output_files']['value'].split(', ')

In [82]:
output_files

['https://w3id.org/mint/instance/DSSAT_summary']

In [122]:
def get_variables(file):
    print(file)
    params = (
        ('io', file),
        ('endpoint', 'https://endpoint.mint.isi.edu/ds/query'),
    )

    response = requests.get('https://query.mint.isi.edu/api/mintproject/MINT-ModelCatalogQueries/getI_OVariablesAndUnits', params=params)
    variables = response.json()['results']['bindings']

    variables_table = []
    for v in variables:
        e = {'Label': v.get('label',{}).get('value'),
             'Long Name': v.get('longName',{}).get('value'),
             'Description': v.get('description',{}).get('value'),
             'Standard Name': v.get('sn',{}).get('value'),
             'Unit': v.get('unit',{}).get('value')}
        variables_table.append(e)
    df = pd.DataFrame.from_records(variables_table)[['Label','Long Name','Description','Standard Name','Unit']]
    return df

## DSSAT Inputs by File
Now we can show the variables for each input file

In [114]:
get_variables(input_files[0])

https://w3id.org/mint/instance/DSSAT_weatherFile


,Label,Long Name,Description,Standard Name,Unit
0,WSTA,Weather station ID,Weather station identifier to link to site inf...,station~weather__identification_number,None
1,LAT,Weather station latitude,Latitude of station- degrees decimal with Nort...,station~weather__latitude,decimal degrees
2,TAV,mean annual temperature,Temperature average for whole year,air__yearly_average_of_temperature,degC
3,TMIN,minimum air temperature,Temperature of air- minimum,atmosphere_air__daily_minimum_of_temperature,degC
4,TMAX,maximum air temperature,Temperature of air- maximum,atmosphere_air__daily_maximum_of_temperature,degC
5,LONG,Weather station longitude,Longitude of station- degrees decimals with Ea...,station~weather__longitude,decimal degrees
6,SRAD,Solar radiation,Solar radiation,atmosphere_radiation~solar__energy_flux,MJ m-2 d-1
7,RAIN,precipitation,Rainfall (including moisture in snow) in one day,atmosphere_water__one-day_time_integral_of_pre...,mm day-1


In [115]:
get_variables(input_files[1])

https://w3id.org/mint/instance/DSSAT_simulationFile


,Label,Long Name,Description,Standard Name,Unit
0,FDATE,fertilization_date,Fertilization date,soil__fertilization_date,date
1,PPOP,planting density,Plant population at planting,crop__count-per-area_planting_density,number m-2
2,SNO3,initial nitrate,Initial NO3 conc. as elemental N on dry wt. ba...,soil_nitrate-as-nitrogen~elemental__dry_mass_f...,ppm
3,FDEP,fertilizer depth,Fertilizer application depth,soil_fertilizer__application_depth,cm
4,INGENO,cultivar id,Cultivar or line or genotype identifier,cultivar__line-or-genotype_identification_code,code
5,PLRS,planting row spacing,Row spacing,crop_row__planting_spacing,cm
6,PDATE,planting date,Planting or sowing date,crop__planting_or_sowing_date,date
7,PLDP,planting depth,Planting depth,crop__planting_depth,mm
8,CR,crop id,Crop (or weed) species identifier,crop-or-weed__species_identification_code,code
9,YCRD,field latitude,Field latitude,field__latitude,decimal degrees


In [116]:
get_variables(input_files[2])

https://w3id.org/mint/instance/DSSAT_sol


,Label,Long Name,Description,Standard Name,Unit
0,SSKS,Saturated hydraulic conductivity,Saturated hydraulic conductivity,soil_water__saturated_hydraulic_conductivity,cm h-1
1,SLB,Soil layer base depth,Soil layer base depth,soil_layer~base__depth,cm
2,SALB,albedo,Soil albedo- dry soil,land_soil~dry_surface__albedo,None
3,SLLL,soil lower limit,Soil water- lower limit,soil_water__lower_limit_of_volume_fraction,cm3 cm-3
4,SLCL,Soil clay percentage,Soil texture- clay (<0.002 mm) weight percent ...,soil_clay__mass_fraction,%
5,SDUL,soil drained upper limit,Soil water- drained upper limit,soil_water__upper_limit_of_drained_volume_frac...,cm3 cm-3
6,SLSI,Soil silt percentage,Soil texture- silt (0.05 to 0.002 mm) weight p...,soil_silt__mass_fraction,%
7,SLOC,soil organic carbon,Total soil organic carbon by layer,soil_matter~organic__mass_fraction,g 100g-1
8,SSAT,soil water content at saturation,Soil water- saturated,soil_water__saturated_volume_fraction,cm3 cm-3
9,SBDM,soil bulk density,Soil bulk density when moist for layer,soil~dry__bulk_mass-per-volume_density,g cm-3


## DSSAT Outputs by File
And we can also show the variables for each output file

In [123]:
get_variables(output_files[0])

https://w3id.org/mint/instance/DSSAT_summary


,Label,Long Name,Description,Standard Name,Unit
0,CWAM,tops dry weigth,Tops dry weight at maturity,crop_tops__maturity_dry_mass-per-area_yield,kg ha-1
1,HWAM,harvest yield,Harvest yield at harvest maturity (dry wt),crop__maturity_dry_harvested_mass-per-area_yield,kg ha-1
